In [1]:
import numpy as np
import pandas as pd
import ncafm as nc
import ncafm.io as io
import ncafm.functions as fcn

import matplotlib.pyplot as plt

In [57]:
data_path = nc.io.get_example_data_file_path('all_data_dataframe_237K.csv', data_dir='ncafm/example_data')

In [58]:
data_dataframe = nc.io.load_data(data_path)

In [59]:
data_dataframe

,z,-0.0,0.75,2.0,4.0,6.0,8.0,10.0
0,-3.182825e-08,51.750179,51.581310,51.130814,49.505810,46.584351,41.749966,33.953133
1,-3.162825e-08,43.953426,43.928509,43.816441,42.090847,39.372910,34.885902,27.489771
2,-3.142825e-08,36.527794,36.373009,36.233540,34.562767,31.635241,27.508020,20.527580
3,-3.122825e-08,28.104694,28.243338,28.223377,26.402557,23.151896,18.147322,11.085422
4,-3.102825e-08,19.526001,19.727394,19.668325,17.515757,13.588511,7.788128,-0.174825
...,...,...,...,...,...,...,...,...
108,-1.022831e-08,-0.629711,-0.526609,-0.487231,-1.042318,-2.116411,-3.999531,-6.450719
109,-1.002831e-08,-0.676826,-0.554232,-0.562571,-0.982578,-2.094154,-4.020424,-6.403704
110,-9.828311e-09,-0.634787,-0.559015,-0.483544,-1.015127,-2.147274,-3.872620,-6.350922
111,-9.628311e-09,-0.721081,-0.548445,-0.548138,-1.033069,-2.181116,-3.927863,-6.343535


In [52]:
a = 0.5*10**-9
k = 1800
f_0 = 19796.5

z_real = data_dataframe['z'] - data_dataframe['z'][0] +1*10**(-9)

In [55]:
z_real.to_numpy().shape

(113,)

In [62]:
force_0p75 = fcn.calculate_force(a,k,f_0, z_real, df_dataframe['0.75'])

In [ ]:
plt.scatter(z_real[:-1]*10**9, lj.calculate_force(a,k,f_0, z_real, df_dataframe['0.75'])*10**9, s= 4, label = '0.75 V')
plt.ylabel('force (nN)')
plt.xlabel('tip height (nm)');

In [ ]:
plt.scatter(z_real[:-1]*10**9, lj.calculate_force(a,k,f_0, z_real, df_dataframe['0.75'])*10**9, s= 4, label = '0.75 V')

plt.scatter(z_real[:-1]*10**9, lj.calculate_force(a,k,f_0, z_real, df_dataframe['6.0'])*10**9, s= 4, label = '6.00 V')

plt.scatter(z_real[:-1]*10**9, lj.calculate_force(a,k,f_0, z_real, df_dataframe['10.0'])*10**9, s= 4, label = '10.00 V')

plt.ylabel('force (nN)')
plt.xlabel('tip height (nm)')
plt.legend();

In [ ]:
plt.scatter(z_real*10**9, df_dataframe['0.75'], s= 4, label = '0.75 V')

plt.scatter(z_real*10**9, df_dataframe['6.0'], s= 4, label = '6.00 V')

plt.scatter(z_real*10**9, df_dataframe['10.0'], s= 4, label = '10.00 V')

plt.ylabel('frequency shift (Hz)')
plt.xlabel('tip height (nm)')

plt.legend();

In [ ]:
upper = lj.calculate_force(a,k,f_0, z_real, df_dataframe['0.75']+0.5)*10**9
lower = lj.calculate_force(a,k,f_0, z_real, df_dataframe['0.75']-0.5)*10**9

In [ ]:
error = (upper-lower)/2

In [ ]:
plt.plot(error)